In [ ]:
import sys
sys.path.append('./styles')
from init_style import init
init()

<!-- Header banner -->
<div class="banner"><div>OpenAD Demonstration</div><b>OpenAD<span>Tutorial</span></b></div>

# openad-OpenToolkit Demonstration

## Working with OpenAD Magic Commands

When using Magic commands to access the Openad toolkit you have 2 options 

1. `%openad` provides a simple user interface that provides styled and formatted objects back to the notebook. Tables use pandas Dataframe Styler object. These can be converted back to data frame objects using `.data` on the object or using the in memory assistant which will copy the last result to a file , dataframe or to the dataviewer.
  When this is available you will see `Next up, you can run: result open/edit/copy/display/as dataframe/save [as '<filename.csv>']` in the output.
  
  This magic command is the recommended version to use as it willprovide all warning and results visually.
  
2. `%openadd` is the second form that allows you to return api style results in dataframe or list formats that can be used programatically for functions or flows in your notebook. This is good for prebuilt notebook process flows.

# ***What Commands Are Available  ?***

To learn what commands available you simply need to enter a `?` at the command line or at a magic command to list avialable commands

In [2]:
%openad ?

## Available Commands - Main

Macromolecules: <br> 
`show mmol|protein <fasta> | '<pdb_id>'` <br> 

General: <br> 
`openad` <br> 
`get status` <br> 
`display history ` <br> 
`clear sessions` <br> 

Workspaces: <br> 
`set workspace <workspace_name>` <br> 
`get workspace [ <workspace_name> ]` <br> 
`create workspace <workspace_name> [ description('<description>') on path '<path>' ]` <br> 
`remove workspace <workspace_name> ` <br> 
`list workspaces` <br> 

Toolkits: <br> 
`ds4sd` <br> 
`rxn` <br> 
`list toolkits` <br> 
`list all toolkits` <br> 
`add toolkit <toolkit_name>` <br> 
`remove toolkit <toolkit_name>` <br> 
`update toolkit <toolkit_name>` <br> 
`update all toolkits` <br> 
`set context <toolkit_name> [ reset ]` <br> 
`get context` <br> 
`unset context` <br> 

Runs: <br> 
`create run` <br> 
`remove run <run_name>` <br> 
`save run as <run_name>` <br> 
`run <run_name>` <br> 
`list runs` <br> 
`display run <run_name>` <br> 

Utility: <br> 
`display data '<filename.csv>'` <br> 
`  -> result save [as '<filename.csv>']` <br> 
`  -> result open` <br> 
`  -> result edit` <br> 
`  -> result copy` <br> 
`  -> result display` <br> 
`  -> result as dataframe` <br> 

GUI: <br> 
`install gui` <br> 
`launch gui` <br> 
`restart gui` <br> 
`quit gui` <br> 

LLM: <br> 
`tell me <how to do xyz>` <br> 
`set llm  <language_model_name>` <br> 
`clear llm auth` <br> 

File System: <br> 
`list files [ path ]` <br> 
`import from '<external_source_file>' to '<workspace_file>'` <br> 
`export from '<workspace_file>' to '<external_file>'` <br> 
`copy file '<workspace_file>' to '<other_workspace_name>'` <br> 
`remove '<filename>'` <br> 
`open '<filename>'` <br> 

Help: <br> 
`intro` <br> 
`docs` <br> 
`?` <br> 
`? ...`   --> List all commands containing "..." <br> 
`... ?`   --> List all commands starting with "..." <br> 

Model: <br> 
`model auth list` <br> 
`model auth add group '<auth_group>'|<auth_group> with '<api_key>'` <br> 
`model auth remove group '<auth_group>' | <auth_group>` <br> 
`model auth add service '<service_name>'|,service_name> to group '<auth_group>'|<auth_group>` <br> 
`model auth remove service '<service_name>'|<service_name>` <br> 
`model service status` <br> 
`model service describe '<service_name>'|<service_name>` <br> 
`model catalog list` <br> 
`uncatalog model service '<service_name>'|<service_name>` <br> 
`catalog model service from (remote) '<path> or <github> or <service_url>' as  '<service_name>'|<service_name>   USING (<parameter>=<value> <parameter>=<value>)` <br> 
`model service up '<service_name>'|<service_name> [no_gpu]}` <br> 
`model service local up '<service_name>'|<service_name> ` <br> 
`model service down '<service_name>'|<service_name>` <br> 
`get model service '<service_name>'|<service_name> result '<result_id>' ` <br> 

Small Molecules: <br> 
`display molecule|mol <name> | <smiles> | <inchi> | <inchikey> | <cid>` <br> 
`show molecule|mol <name> | <smiles> | <inchi> | <inchikey> | <cid>` <br> 
`show molset|molecule-set '<molset_or_sdf_or_smi_path>' | using dataframe <dataframe>` <br> 
`@(<name> | <smiles> | <inchi> | <inchikey> | <cid>)>><molecule_property_name>` <br> 
`export molecule|mol <name> | <smiles> | <inchi> | <inchikey> | <cid> [ as file ]` <br> 

Molecule Working Set: <br> 
`add molecule|mol <name> | <smiles> | <inchi> | <inchikey> | <cid> [ as '<name>' ] [ basic ] [ force ]` <br> 
`remove molecule|mol <name> | <smiles> | <inchi> | <inchikey> | <cid> [ force ]` <br> 
`list molecules|mols` <br> 
`show molecules|mols` <br> 
`enrich molecules|mols with analysis` <br> 
`clear analysis cache` <br> 
`display sources <name> | <smiles> | <inchi> | <inchikey> | <cid>` <br> 
`rename molecule|mol <molecule_identifer_string> as <molecule_name>` <br> 
`load molecules|mols from file '<filename.molset.json|sdf|csv|smi>' [ enrich ] [ append ]` <br> 
`load molecules|mols from dataframe <dataframe> [ enrich ] [ append ]` <br> 
`merge molecules|mols data from dataframe <dataframe> [ enrich ]` <br> 
`export molecules|mols [ as '<filename.molset.json|sdf|csv|smi>' ]` <br> 
`clear molecules|mols [ force ]`



## Available Commands - DS4SD

Search Molecules: <br> 
`search for similar molecules to '<smiles>' [ save as '<filename.csv>' ]` <br> 
`search for molecules in patents from list ['<patent1>', '<patent2>', ...] | dataframe <dataframe_name> | file '<filename.csv>' [ save as '<filename.csv>' ]` <br> 
`search for patents containing molecule '<smiles>' | '<inchi>' | '<inchikey>' [ save as '<filename.csv>' ]` <br> 
`search for substructure instances of '<smiles>' [ save as '<filename.csv>' ]` <br> 

Search Collections: <br> 
`search collection '<collection_name_or_key>' for '<search_string>' [ using (page_size=<int> system_id=<system_id> edit_distance=<integer> display_first=<integer>) ] show (data | docs) [ estimate only | return as data | save as '<filename.csv>' ]` <br> 
`display collection matches for '<search_string>' [ save as '<filename.csv>' ]` <br> 

Collections: <br> 
`display collections in domains from list <list_of_domains> [ save as '<filename.csv>' ] ` <br> 
`display all collections [ save as '<filename.csv>' ]` <br> 
`display collections for domain '<domain_name>' ` <br> 
`display collection details '<collection_name_or_key>'` <br> 

 i  <span style="color: #ccc">To learn more about the DS4SD toolkit, run `ds4sd`.</span> <br> 


# ***Individual Toolkits Help***

To look at the help purely for a specific toolkit, you simply need to ensure the toolkit is added then run the following.

%openad add toolkit rxn

In [3]:
%openad ? rxn

## Available Commands - RXN

General: <br> 
`interpret recipe '<recipe_paragraph>' | '<txt_filename>' ` <br> 
`list rxn models` <br> 

Retrosynthesis: <br> 
`predict retrosynthesis '<smiles>' [ using (<parameter>=<value> <parameter>=<value>) ]` <br> 

Prediction: <br> 
`predict reaction in batch from dataframe <dataframe_name> | file '<filename.csv>' | list ['<smiles>.<smiles>','<smiles>.<smiles>'] [ using (ai_model='<ai_model>') ] [ use_saved ]` <br> 
`predict reaction '<smiles>.<smiles>' [ using (ai_model='<ai_model>') ] [ use_saved ]` <br> 
`predict reaction topn in batch from dataframe <dataframe_name> | file '<filename.csv>' | list ['<smiles>.<smiles>','<smiles>.<smiles>'] [ using (topn=<integer> ai_model='<ai_model>') ] [ use_saved ]` <br> 

 i  <span style="color: #ccc">To learn more about the RXN toolkit, run `rxn`.</span> <br> 


In [4]:
%openad clear sessions
%openad remove toolkit ds4sd
%openad remove toolkit rxn
%openad add toolkit ds4sd
%openad add toolkit rxn
%openad set context ds4sd

<span style="color: #dc0">Terminate all other sessions? <br> 
<span style="color: #ccc">Make sure you don't have any crucial processes running in other windows</span></span> <br> 


(y/n):  y


<span style="color: #090">All sessions have been cleared</span> <br> 


<span style="color: #090">The <span style="color: #dc0">DS4SD</span> toolkit was removed</span> <br> 


<span style="color: #090">The <span style="color: #dc0">RXN</span> toolkit was removed</span> <br> 


<span style="color: #090">You successfully logged in to <span style="color: #dc0">DS4SD</span></span> <br> 
<span style="color: #ccc">Your access token expires on Wed Dec 11, 2024  at 20:55</span> <br> 


<span style="color: #090">The <span style="color: #dc0">DS4SD</span> toolkit was successfully installed</span> <br> 
<span style="color: #ccc">To see what you can do, run `? ds4sd`</span> <br> 


<span style="color: #090">You successfully logged in to <span style="color: #dc0">RXN</span></span> <br> 
<span style="color: #ccc">Your access token does not have an expiration date</span> <br> 


<span style="color: #090">The <span style="color: #dc0">RXN</span> toolkit was successfully installed</span> <br> 
<span style="color: #ccc">To see what you can do, run `? rxn`</span> <br> 


<span style="color: #090">You successfully logged in to <span style="color: #dc0">DS4SD</span></span> <br> 
<span style="color: #ccc">Your access token expires on Wed Dec 11, 2024  at 20:55</span> <br> 


# ***Working Within the Open ToolKit Environment***

There are a Range of different commands that can be run to navigate through the toolkit, looking at `workspaces`, `toolkits`, `files` and `runs` within `workspaces` etc.<br> For example:

In [5]:
%openad list workspaces

<span style="color: #ccc">To see what you can do with a workspace, run `? workspace`.</span> <br> 


Workspace,Description
* DEFAULT,This is your default workspace.


In [6]:
%openad list files 

File Name,Size,Last Edited
_mols/,None,None
_result_cache/,None,None
_runs/,None,None
a/,None,None
carbohydrates/,None,None
nucleic acid/,None,None
proteins/,None,None
-,-,-
"2,4-dimethylpentane.mol.json",34.26 kB,"Mon Sep 16, 2024 - 17:15"
3-methylpentane.mol.json,32.79 kB,"Mon Sep 16, 2024 - 17:15"


# ***Runs***

The concept of a run is to be able to re execute a consecutive set of commands in order, ideally that provide information to one another.

In [1]:
%openad ? run

TypeError: sequence item 0: expected str instance, Markdown found

We are now going to create a run for demostration purposes.

In [ ]:
%openad create run

In [ ]:
%openad list all toolkits

In [ ]:
%openad set context rxn
%openad List rxn models

In [ ]:
%openad list runs

When we want to stop and record the run we simply execute the following.

In [ ]:
%openad save run as test_run3

We can display its contents with the following command.

In [ ]:
%openad display run test_run3

Now lets run it !

In [ ]:
%openad run test_run3

In [ ]:
%openad set llm bam

# ***Deep Search to RXN Demonstration***

In this Demonstration we are going to Search for a Smiles definition of a Drug in <b>Deep Search</b> and determine retrosynthesis Paths in <b>RXN</b>

First lets ask the `Tell Me` Me Functionality how we should do it.

In [ ]:
%openad tell me how to get the canonical smiles string for a molecule  and then what command to predict retrosynthesis

In [ ]:
mol = %openad @aspirin>>canonical_smiles

%openad set context ds4sd
%openad search for patents containing molecule '{mol}'

%openad set context rxn
%openad predict retrosynthesis '{mol}'

## ***Now lets Do a sequence of different commands and look at the results***

### ***Set the Tool Context to Deep Search***

In [ ]:
%openad set context ds4sd
target_smile= %openad @tritace>>canonical_smiles

In [ ]:
%openad search for patents containing molecule '{target_smile}'

### ***Let's look at what Domains and Collections exist for us to search***

In [ ]:
%openad  display all collections 

## Let's Look at The Collection we may find the most interesting

In [ ]:
%openad  display collection details 'Crossref'

Now we will Search for collections with a mention of the Molecule called  `Sibofimloc`, research it and find its SMILE returning Result to a data frame

What collections mention the molecule  'Sibofimloc' ?

In [ ]:
%openad  display collection matches for 'Sibofimloc'

### Lets Research the Drug for Safety and read the Papers where it is discused

In [ ]:
%openad search collection 'crossref' for 'Sibofimloc AND safety'    SHOW (docs data) 

### Lets Pull the Data for the Molecule <br>
by using the molecules Functions

In [ ]:
%openad display molecule Sibofimloc

In [ ]:
target_string=%openad @Sibofimloc>>canonical_smiles

### Let's have a look at similar molecules and Related Patents<br>

This will allows us also to look at similar targets we may want to pursue if our further experiements do not work as intended.

In [ ]:
similar_targets = %openadd search for similar molecules to '{target_string}' save as 'similar_targets.csv'
display(similar_targets)

In [ ]:
%openad load molecules using dataframe similar_targets
%openad show molecules 

In [ ]:

%openad search for patents containing molecule '{target_string}'

For each of those similar targets lets go and find the patents that they are mentioned in !

In [ ]:

for similar_mol  in similar_targets['SMILES']:
   my_patents=%openadd search for patents containing molecule '{similar_mol}'
   %openad search for molecules in patents from dataframe  my_patents

In [ ]:
%openad display molecule CCC(C)C(=O)N1CCC2(CC1)c1cc(C#CC3OC(CO)C(O)C(O)C3O)ccc1-c1ccc(C#CC3OC(CO)C(O)C(O)C3O)cc12

### ***Set Context to the RXN Toolkit and run Retro Synthesis Prediction On our chosen Molecule***

Now with our Original target Molecule `Sibofimloc` lets perform retrosynthesis analyis to determine routes to snythesize the molecule.

In [ ]:
%openad set context rxn
%openad predict retrosynthesis '{target_string}' using( max_steps=3 )

In [ ]:
%openad add molecule {target_string}
%openad enrich molecules with analysis
%openad display molecule {target_string}

## Interpreting  Recipes

In [ ]:
%openad interpret recipe 'To a stirred solution of 7-(difluoromethylsulfonyl)-4-fluoro-indan-1-one (110 mg, 0.42 mmol)\
in methanol (4 mL) was added sodium borohydride \
(24 mg, 0.62 mmol). The reaction mixture was stirred at ambient temperature for 1 hour.'

## Predicting Reaction Outcomes

### Single Predictions

In [ ]:
%openad  predict reaction 'Cl.CCC(=O)NCCC.O' use_saved

In [ ]:
%openad import  from '~/openad_notebooks/examples/reactions.csv' to 'reactions.csv'
%openad predict reaction in batch from file 'reactions.csv'

### Batch Predictions

In [ ]:
%openad predict reaction in batch from list ['BrBr.c1ccc2cc3ccccc3cc2c1CCO' ,'Cl.CCC(=O)NCCC.O'] 

### Using the [use_saved] clause

Providing the ability to reuse recently run previous reaction runs

In [ ]:
%openad predict reaction in batch  from list ['BrBr.c1ccc2cc3ccccc3cc2c1CCO' ,'BrBr.c1ccc2cc3ccccc3cc2c1','Cl.CCC(=O)NCCC.O'] use_saved

In [ ]:
%openad set context rxn
import pandas
lst = ['BrBr.c1ccc2cc3ccccc3cc2c1CCO' ,'Cl.CCC(=O)NCCC.O'] 
batch_list = pandas.DataFrame(lst,columns=['reactions'])
%openad predict reaction topn in batch from dataframe batch_list using (topn=3) use_saved

## Quickly investingation and adding to a set of molecules

In [ ]:
%openad add molecule tritace force
%openad add molecule aspirin force
%openad add molecule Ibuprofen force
%openad add molecule entresto force

In [ ]:
%openad list molecules

In [ ]:
mol = %openad @tritace>>canonical_smiles
%openad set context rxn
%openad predict retrosynthesis '{mol}' using( max_steps=3 )

In [ ]:
%openad set context ds4sd
%openad search for patents containing molecule '{mol}'
%openad search for similar molecules to '{mol}'


In [ ]:
%openad list molecules

In [ ]:
%openad enrich molecules with analysis 
%openad display molecule tritace

In [ ]:
%openad list molecules